# Data preprocessing

In [1]:
pwd

'C:\\Users\\quay\\Documents\\ShareIt\\notebooks'

In [2]:
cd ..\data\raw

C:\Users\quay\Documents\ShareIt\data\raw


## Imports

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import secrets
import string
from pathlib import Path
import re

## Reading files

In [4]:
users = pd.read_csv("amazon_prime_users.csv")

In [5]:
items = pd.read_csv("ElectronicsData.csv")

## Data cleaning

### *users Schema*

In [6]:
users.head()

,User ID,Name,Email Address,Username,Date of Birth,Gender,Location,Membership Start Date,Membership End Date,Subscription Plan,Payment Information,Renewal Status,Usage Frequency,Purchase History,Favorite Genres,Devices Used,Engagement Metrics,Feedback/Ratings,Customer Support Interactions
0,1,Ronald Murphy,williamholland@example.com,williamholland,1953-06-03,Male,Rebeccachester,2024-01-15,2025-01-14,Annual,Mastercard,Manual,Regular,Electronics,Documentary,Smart TV,Medium,3.6,3
1,2,Scott Allen,scott22@example.org,scott22,1978-07-08,Male,Mcphersonview,2024-01-07,2025-01-06,Monthly,Visa,Manual,Regular,Electronics,Horror,Smartphone,Medium,3.8,7
2,3,Jonathan Parrish,brooke16@example.org,brooke16,1994-12-06,Female,Youngfort,2024-04-13,2025-04-13,Monthly,Mastercard,Manual,Regular,Books,Comedy,Smart TV,Low,3.3,8
3,4,Megan Williams,elizabeth31@example.net,elizabeth31,1964-12-22,Female,Feliciashire,2024-01-24,2025-01-23,Monthly,Amex,Auto-renew,Regular,Electronics,Documentary,Smart TV,High,3.3,7
4,5,Kathryn Brown,pattersonalexandra@example.org,pattersonalexandra,1961-06-04,Male,Port Deborah,2024-02-14,2025-02-13,Annual,Visa,Auto-renew,Frequent,Clothing,Drama,Smart TV,Low,4.3,1


In [7]:
users.isna().sum().sum()

0

In [8]:
users.shape

(2500, 19)

#### Change columns to PascalCase 

In [9]:
users.columns = users.columns.str.replace(' ', '')

In [10]:
users.columns

Index(['UserID', 'Name', 'EmailAddress', 'Username', 'DateofBirth', 'Gender',
       'Location', 'MembershipStartDate', 'MembershipEndDate',
       'SubscriptionPlan', 'PaymentInformation', 'RenewalStatus',
       'UsageFrequency', 'PurchaseHistory', 'FavoriteGenres', 'DevicesUsed',
       'EngagementMetrics', 'Feedback/Ratings', 'CustomerSupportInteractions'],
      dtype='object')

In [11]:
users[['FirstName', 'LastName']] = users['Name'].str.split(' ', n = 1, expand=True)

In [12]:
users2 = users[["UserID", "FirstName", "LastName", "EmailAddress"]].copy()

In [13]:
users2 = users2.rename(columns={'EmailAddress': 'Email'})

In [14]:
users2.head()

,UserID,FirstName,LastName,Email
0,1,Ronald,Murphy,williamholland@example.com
1,2,Scott,Allen,scott22@example.org
2,3,Jonathan,Parrish,brooke16@example.org
3,4,Megan,Williams,elizabeth31@example.net
4,5,Kathryn,Brown,pattersonalexandra@example.org


In [15]:
users2.isna().sum().sum()

0

In [16]:
def generate_unique_phone_numbers(count):
    phone_numbers = set()
    while len(phone_numbers) < count:
        start_digit = np.random.choice([8, 9])
        
        remaining_digits = np.random.randint(0, 10, 7)
        phone_number = str(start_digit) + ''.join(map(str, remaining_digits))
        phone_numbers.add(phone_number)
    return list(phone_numbers)

num_records = len(users)
unique_phone_numbers = generate_unique_phone_numbers(num_records)

In [17]:
users2['Contact'] = unique_phone_numbers

In [18]:
users2

,UserID,FirstName,LastName,Email,Contact
0,1,Ronald,Murphy,williamholland@example.com,90897989
1,2,Scott,Allen,scott22@example.org,94646457
2,3,Jonathan,Parrish,brooke16@example.org,83973507
3,4,Megan,Williams,elizabeth31@example.net,99840535
4,5,Kathryn,Brown,pattersonalexandra@example.org,92236075
...,...,...,...,...,...
2495,2496,Michael,Lopez,williamsroberto@example.org,98942566
2496,2497,Matthew,Woodard,lkaiser@example.com,85886311
2497,2498,Morgan,Barnes,erikaholland@example.net,94351611
2498,2499,Gina,Castaneda,reedcourtney@example.net,98752554


In [19]:
users2['Contact'].nunique()

2500

In [20]:
def generate_password(length=10):
    characters = string.ascii_letters + string.digits
    return ''.join(random.choice(characters) for i in range(length))



In [21]:
users2['Password'] = users.apply(lambda x: generate_password(), axis=1)

In [22]:
users2 = users2[["UserID", "FirstName", "LastName", "Email", "Password", "Contact"]]

In [23]:
users2

,UserID,FirstName,LastName,Email,Password,Contact
0,1,Ronald,Murphy,williamholland@example.com,kavjPzOu28,90897989
1,2,Scott,Allen,scott22@example.org,cbEjpBw0Yu,94646457
2,3,Jonathan,Parrish,brooke16@example.org,kjvrprWxDu,83973507
3,4,Megan,Williams,elizabeth31@example.net,Mb2mtsJ21m,99840535
4,5,Kathryn,Brown,pattersonalexandra@example.org,OfKc33Q7sf,92236075
...,...,...,...,...,...,...
2495,2496,Michael,Lopez,williamsroberto@example.org,tBD1Zhxh7g,98942566
2496,2497,Matthew,Woodard,lkaiser@example.com,FMe4QBlRrs,85886311
2497,2498,Morgan,Barnes,erikaholland@example.net,QQxmqJZMou,94351611
2498,2499,Gina,Castaneda,reedcourtney@example.net,Of9Jj88rRx,98752554


In [58]:
users2.isna().sum().sum()

0

### Unit test

In [24]:
def is_valid_email(email: str) -> bool:
    email_pattern = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    return bool(re.match(email_pattern, email))

def test_email_format(df):
    invalid_emails = df[~users2['Email'].apply(is_valid_email)]
    assert invalid_emails.empty, f"Invalid emails found: {invalid_emails['Email'].tolist()}"
    print("all emails valid")
test_email_format(users2)

all emails valid


### *items Schema*

In [25]:
items.head()

,Sub Category,Price,Discount,Rating,Title,Currency,Feature
0,Batteries,$20.99,No Discount,NaN,"Duracell Coppertop Alkaline AA Batteries, 40-c...",$,POWER BOOST™ Ingredients Guaranteed For 12 Y...
1,Batteries,$20.99,No Discount,NaN,"Duracell Coppertop Alkaline AAA Batteries, 40-...",$,POWER BOOST™ Ingredients Guaranteed For 12 Y...
2,Batteries,$20.99,No Discount,NaN,"Duracell 9V Alkaline Batteries, 8-count",$,Long-Lasting Batteries Designed for Dependabil...
3,Batteries,$18.99,No Discount,NaN,"Duracell C Alkaline Batteries, 14-count",$,Long-Lasting Batteries Designed for Dependabil...
4,Batteries,$14.99,No Discount,Rated 4.8 out of 5 stars based on 125 reviews.,"Duracell 2032 Lithium Coin Batteries, 12-count",$,Our Longest Lasting Lithium Coin Battery Ever ...


In [26]:
items.isna().sum().sum()

14

In [27]:
items = items.dropna().reset_index(drop=True)

In [28]:
items.isna().sum().sum()

0

In [29]:
items.head()

,Sub Category,Price,Discount,Rating,Title,Currency,Feature
0,Batteries,$14.99,No Discount,Rated 4.8 out of 5 stars based on 125 reviews.,"Duracell 2032 Lithium Coin Batteries, 12-count",$,Our Longest Lasting Lithium Coin Battery Ever ...
1,Cameras & Camcorders,$289.99,Price valid through 1/21/24,Rated 4.2 out of 5 stars based on 238 reviews.,GoPro HERO11 Black Action Camera Bundle,$,"Also Includes: Shorty Hand Grip, Spare Enduro ..."
2,Cameras & Camcorders,$399.99,Price valid through 3/31/24,Rated 4.4 out of 5 stars based on 99 reviews.,Insta360 X3 - 5.7K 360° Action Camera Adventur...,$,X3 Captures 5.7K 360 Active HDR Video Dual 1...
3,Cameras & Camcorders,$429.99,No Discount,Rated 4.4 out of 5 stars based on 115 reviews.,DJI Mini 3 Drone Aerial Camera Bundle,$,Less Than 249g (0.55lbs) Ultralight and Foldab...
4,Cameras & Camcorders,$239.97,No Discount,Rated 4.1 out of 5 stars based on 311 reviews.,GoPro HERO10 Black Action Camera Bundle,$,"Also Includes: Extra Battery/Charger, Compact ..."


In [30]:
items2 = items[['Title', 'Feature', 'Price']].copy()

In [31]:
items2.rename(columns={'Title': 'ItemName', 'Feature': 'Description'}, inplace=True)

In [32]:
items2.head()

,ItemName,Description,Price
0,"Duracell 2032 Lithium Coin Batteries, 12-count",Our Longest Lasting Lithium Coin Battery Ever ...,$14.99
1,GoPro HERO11 Black Action Camera Bundle,"Also Includes: Shorty Hand Grip, Spare Enduro ...",$289.99
2,Insta360 X3 - 5.7K 360° Action Camera Adventur...,X3 Captures 5.7K 360 Active HDR Video Dual 1...,$399.99
3,DJI Mini 3 Drone Aerial Camera Bundle,Less Than 249g (0.55lbs) Ultralight and Foldab...,$429.99
4,GoPro HERO10 Black Action Camera Bundle,"Also Includes: Extra Battery/Charger, Compact ...",$239.97


In [33]:
items2['Price'] = items2['Price'].str.replace('$', '')
items2['Price'] = items2['Price'].str.replace(',', '')

In [34]:
items2['Price'] = pd.to_numeric(items2['Price'], errors='coerce')
items2 = items2.dropna(subset=['Price'])
items2['Price'] = items2['Price'].round(2)

In [35]:
items2.isna().sum().sum()

0

In [36]:
items2.shape

(621, 3)

In [37]:
items2['ItemID'] = range(1, len(items2) + 1)

In [38]:
items2.head()

,ItemName,Description,Price,ItemID
0,"Duracell 2032 Lithium Coin Batteries, 12-count",Our Longest Lasting Lithium Coin Battery Ever ...,14.99,1
1,GoPro HERO11 Black Action Camera Bundle,"Also Includes: Shorty Hand Grip, Spare Enduro ...",289.99,2
2,Insta360 X3 - 5.7K 360° Action Camera Adventur...,X3 Captures 5.7K 360 Active HDR Video Dual 1...,399.99,3
3,DJI Mini 3 Drone Aerial Camera Bundle,Less Than 249g (0.55lbs) Ultralight and Foldab...,429.99,4
4,GoPro HERO10 Black Action Camera Bundle,"Also Includes: Extra Battery/Charger, Compact ...",239.97,5


In [39]:
items2['UserID'] = np.random.randint(1, 2501, len(items2))

In [40]:
items2.head()

,ItemName,Description,Price,ItemID,UserID
0,"Duracell 2032 Lithium Coin Batteries, 12-count",Our Longest Lasting Lithium Coin Battery Ever ...,14.99,1,394
1,GoPro HERO11 Black Action Camera Bundle,"Also Includes: Shorty Hand Grip, Spare Enduro ...",289.99,2,262
2,Insta360 X3 - 5.7K 360° Action Camera Adventur...,X3 Captures 5.7K 360 Active HDR Video Dual 1...,399.99,3,2070
3,DJI Mini 3 Drone Aerial Camera Bundle,Less Than 249g (0.55lbs) Ultralight and Foldab...,429.99,4,1030
4,GoPro HERO10 Black Action Camera Bundle,"Also Includes: Extra Battery/Charger, Compact ...",239.97,5,1011


In [41]:
items2 = items2[['ItemID', 'UserID', 'ItemName', 'Description', 'Price']]

In [42]:
items2.head()

,ItemID,UserID,ItemName,Description,Price
0,1,394,"Duracell 2032 Lithium Coin Batteries, 12-count",Our Longest Lasting Lithium Coin Battery Ever ...,14.99
1,2,262,GoPro HERO11 Black Action Camera Bundle,"Also Includes: Shorty Hand Grip, Spare Enduro ...",289.99
2,3,2070,Insta360 X3 - 5.7K 360° Action Camera Adventur...,X3 Captures 5.7K 360 Active HDR Video Dual 1...,399.99
3,4,1030,DJI Mini 3 Drone Aerial Camera Bundle,Less Than 249g (0.55lbs) Ultralight and Foldab...,429.99
4,5,1011,GoPro HERO10 Black Action Camera Bundle,"Also Includes: Extra Battery/Charger, Compact ...",239.97


In [43]:
items2.isna().sum().sum()

0

In [44]:
groupby = items2.groupby('UserID').size().reset_index(name='Count')
groupby[groupby['Count'] > 1].head()

,UserID,Count
0,1,2
5,9,2
8,25,2
19,65,2
28,106,2


### unit test


In [45]:
def test_price(df):
    for index, row in df.iterrows():
        price = row['Price']
        user_id = row['UserID']
        assert pd.api.types.is_numeric_dtype(type(price)), f"UserID {user_id}'s Price {price} is not numeric"
    print("All prices are numeric")
    
test_price(items2)

All prices are numeric


## listings schema

In [46]:
num_entries = 50

# ListingID
listing_id = range(1, num_entries + 1)

# BorrowerID & LenderID
borrower_id = np.random.randint(1, 2501, size=num_entries)
lender_id = np.random.randint(1, 2501, size=num_entries)

for i in range(num_entries):
    while borrower_id[i] == lender_id[i]:
        lender_id[i] = np.random.randint(1, 2501)

# ItemsID
item_ids = np.random.randint(1, 2501, size=num_entries)

# StartDate & EndDate
start_date_base = datetime.strptime('01-01-2024', '%d-%m-%Y')
end_date_limit = datetime.strptime('01-09-2024', '%d-%m-%Y')

start_dates = [start_date_base + timedelta(days=random.randint(0, (end_date_limit - start_date_base).days)) for _ in range(num_entries)]
end_dates = [start_dates[i] + timedelta(days=random.randint(1, (end_date_limit - start_dates[i]).days)) for i in range(num_entries)]

# Creating Return_Flag column with all values as True (1)
return_flag = [1] * num_entries

# Create the DataFrame
listings = pd.DataFrame({
    'ListingID': listing_id,
    'BorrowerID': borrower_id,
    'LenderID': lender_id,
    'ItemID': item_ids,
    'StartDate': start_dates,
    'EndDate': end_dates,
    'ReturnFlag': return_flag
})



In [57]:
listings.isna().sum().sum()

0

In [47]:
listings.head()

,ListingID,BorrowerID,LenderID,ItemID,StartDate,EndDate,ReturnFlag
0,1,2482,316,586,2024-03-17,2024-06-16,1
1,2,2039,1709,912,2024-03-01,2024-08-19,1
2,3,1399,1198,376,2024-07-03,2024-07-11,1
3,4,2315,495,2044,2024-05-20,2024-08-03,1
4,5,1980,19,483,2024-08-01,2024-08-08,1


### small unit test practice

In [48]:
def test_borrower_lender_ids(df):
    for i in range(len(df)):
        assert df.loc[i, 'BorrowerID'] != df.loc[i, 'LenderID'], \
            f"BorrowerID and LenderID are the same for ListingID {df.loc[i, 'ListingID']}"
    print("No same ID")

test_borrower_lender_ids(listings)


No same ID


In [49]:
def test_invalid_dates(df):
    for i in range(len(df)):
        assert df.loc[i, 'EndDate'] > df.loc[i, 'StartDate'], \
            f"EndDate is not greater than StartDate for ListingID {df.loc[i, 'ListingID']}"
    print("No EndDate > StartDate")
test_invalid_dates(listings)

No EndDate > StartDate


# Export CSV

In [50]:
%pwd

'C:\\Users\\quay\\Documents\\ShareIt\\data\\raw'

In [51]:
cd ..

C:\Users\quay\Documents\ShareIt\data


In [52]:
cd clean/

C:\Users\quay\Documents\ShareIt\data\clean


In [53]:
users2.to_csv('users.csv', index=False)
items2.to_csv('items.csv', index=False) 
listings.to_csv('listings.csv', index=False) 